# DAG analysis

Map the genes to their indices so that we can parse the interactions file.

In [1]:
def readGenes(genefile):
    genes = []
    with open(genefile) as f:
        for gene in f:
            genes.append(gene.strip().strip(";").strip('"'))
    return dict([(float(i), gene) for i,gene in enumerate(genes)])
                        

In [2]:
idx2gene = readGenes('GeneIDs.txt')
idx2gene

{0.0: 'rutA',
 1.0: 'rutR',
 2.0: 'nemA',
 3.0: 'fepB',
 4.0: 'rutG',
 5.0: 'phoP',
 6.0: 'rutF',
 7.0: 'rutB',
 8.0: 'rutE',
 9.0: 'arcA',
 10.0: 'ydeA',
 11.0: 'fadL',
 12.0: 'rpsJ',
 13.0: 'yrbL',
 14.0: 'hybC',
 15.0: 'ackA',
 16.0: 'rplB',
 17.0: 'uvrA',
 18.0: 'glnG',
 19.0: 'ddpA',
 20.0: 'ddpF',
 21.0: 'fadI',
 22.0: 'ddpB',
 23.0: 'iraM',
 24.0: 'ddpD',
 25.0: 'yeaG',
 26.0: 'yhdY',
 27.0: 'borD',
 28.0: 'appA',
 29.0: 'ybjG',
 30.0: 'yhdZ',
 31.0: 'potG',
 32.0: 'yeaH',
 33.0: 'ddpX',
 34.0: 'hybO',
 35.0: 'phoQ',
 36.0: 'rutD',
 37.0: 'rplV',
 38.0: 'betB',
 39.0: 'astE',
 40.0: 'hybF',
 41.0: 'betT',
 42.0: 'yhdW',
 43.0: 'tpx',
 44.0: 'metL',
 45.0: 'betA',
 46.0: 'argT',
 47.0: 'astB',
 48.0: 'puuR',
 49.0: 'dcuC',
 50.0: 'potI',
 51.0: 'icd',
 52.0: 'rpsC',
 53.0: 'rutC',
 54.0: 'appC',
 55.0: 'lldP',
 56.0: 'astC',
 57.0: 'purD',
 58.0: 'puuD',
 59.0: 'rstB',
 60.0: 'potF',
 61.0: 'appB',
 62.0: 'betI',
 63.0: 'astA',
 64.0: 'astD',
 65.0: 'rpsS',
 66.0: 'hybG',
 67.0: 

In [3]:
import pandas as pd
import os

ecoli100 = '../data_sets/De-noised_100G_9T_300cPerT_4_DS1'
def parse_master_regulators(file, bins, idx2gene):
    mr = pd.read_csv(os.path.join(ecoli100,'Regs_cID_4.txt'), header=None,
                 names=['Idx'] + [f'bin{i+1}' for i in range(bins)])
    mr['gene'] = [idx2gene[mr.loc[i,'Idx']] for i in mr.index]                                                                     
    return mr.set_index('gene')
mr = parse_master_regulators(ecoli100,9,idx2gene)
mr

,Idx,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,bin9
gene,,,,,,,,,,
slyB,67.0,0.201651,0.954463,0.814158,0.758293,0.808573,0.214715,0.420724,0.498340,0.944724
uvrA,17.0,0.253163,0.239280,0.752575,0.261319,0.206632,0.894642,0.831740,0.476808,0.915584
astC,56.0,0.786113,0.915839,0.475327,0.353435,0.428566,0.478558,0.223267,0.216351,0.977078
metL,44.0,0.767412,0.254815,0.400582,0.915552,0.310638,0.970267,0.466789,0.901425,0.206116
betI,62.0,0.220651,0.447418,0.357020,0.987112,0.792426,0.408655,0.317939,0.746645,0.836442
fadB,93.0,0.786919,0.353563,0.224318,0.985470,0.485508,0.439393,0.281149,0.220886,0.864627
lldD,84.0,0.964934,0.353511,0.781510,0.934178,0.740910,0.867676,0.964883,0.783357,0.244533


In [4]:
def parse_interaction(line,idx2gene):
    regulators = dict()
    K = dict()
    coop_state = dict()
    columns = line.split(',')
    gene = idx2gene[float(columns[0])]
    num_regulators = int(float(columns[1]))
    for i in range(num_regulators):
        regulator_idx = columns[i+2]
        regulator_gene = idx2gene[float(regulator_idx)]
        regulators[regulator_gene] = regulator_idx
        K[regulator_gene] = columns[num_regulators + i + 2]
        coop_state[regulator_gene] = columns[num_regulators*2 + i + 2]
    return dict(gene=gene, regulators=regulators, K=K, coop_state = coop_state)

In [5]:
line = '53.0,5.0,1.0,14.0,67.0,74.0,62.0,1.7310389942765454,2.641368643483525,2.419502251829988,-3.2654552986344143,-3.0023322853469634,2.0,2.0,2.0,2.0,2.0'
parse_interaction(line,idx2gene)

{'gene': 'rutC',
 'regulators': {'rutR': '1.0',
  'hybC': '14.0',
  'slyB': '67.0',
  'ddpC': '74.0',
  'betI': '62.0'},
 'K': {'rutR': '1.7310389942765454',
  'hybC': '2.641368643483525',
  'slyB': '2.419502251829988',
  'ddpC': '-3.2654552986344143',
  'betI': '-3.0023322853469634'},
 'coop_state': {'rutR': '2.0',
  'hybC': '2.0',
  'slyB': '2.0',
  'ddpC': '2.0',
  'betI': '2.0'}}

In [6]:
def parse_interaction_file(interaction_file,idx2gene):
    """input_file_taregts: a csv file, one row per targets. 
       Columns: Target Idx, #regulators, regIdx1,...,regIdx(#regs), K1,...,K(#regs), coop_state1,..., coop_state(#regs)
    """
    interactions = []
    with open(interaction_file) as f:
        for line in f:
            interactions.append(parse_interaction(line,idx2gene))

    return interactions

def interactions2paths(interactions):
    paths = []
    for interaction in interactions:
        for regulator in interaction['regulators']:
            paths.append(f"{regulator}->{interaction['gene']}")
    return paths

def write_dagitty(paths, filename):
    with open(filename, 'w') as out:
        out.write('dag {\n%s\n}' % '\n'.join(paths))
        
write_dagitty(interactions2paths(parse_interaction_file(os.path.join(ecoli100, 'Interaction_cID_4.txt'),
                                     readGenes('GeneIDs.txt'))),
              os.path.join(ecoli100, 'DS1.dagitty'))
                                     
                    
                
            

## run 

In [54]:
!cat DS1.dagitty

dag {
rutR->rutC
hybC->rutC
slyB->rutC
ddpC->rutC
betI->rutC
rutR->argA
hybC->argA
slyB->argA
ddpC->argA
betI->argA
rutR->appC
hybC->appC
slyB->appC
ddpC->appC
betI->appC
rutR->potF
hybC->potF
slyB->potF
ddpC->potF
betI->potF
rutR->potH
hybC->potH
slyB->potH
ddpC->potH
betI->potH
rutR->lldP
hybC->lldP
slyB->lldP
ddpC->lldP
betI->lldP
rutR->rpsC
hybC->rpsC
slyB->rpsC
ddpC->rpsC
betI->rpsC
rutR->argF
hybC->argF
slyB->argF
ddpC->argF
betI->argF
ddpC->pagP
betI->pagP
slyB->treR
ddpC->treR
ddpC->argR
betI->argR
rutR->betT
hybC->betT
rutR->yeaH
hybC->yeaH
rutR->rutD
hybC->rutD
rutR->puuR
hybC->puuR
rutR->rpsJ
hybC->rpsJ
rutR->yhdW
hybC->yhdW
rutR->arcA
hybC->arcA
rutR->rutE
hybC->rutE
rutR->hybO
hybC->hybO
rutR->rplB
hybC->rplB
rutR->iraM
hybC->iraM
rutR->potI
hybC->potI
rutR->potG
hybC->potG
rutR->rplV
hybC->rplV
uvrA->rutR
astC->rutR
metL->rutR
uvrA->hybC
astC->hybC
metL->hybC
rutR->ddpA
hybC->ddpA
rutR->rstB
hybC->rstB
rutR->glnG
hybC->glnG
rutR->rutF
hybC->rutF
rutR->icd
hybC->icd
rutR->

## Read data set 

In [8]:
ds = pd.read_csv(os.path.join(ecoli100, 'simulated_noNoise_0.csv'),index_col=0)
ds.rename(index=idx2gene)

,0,1,2,3,4,5,6,7,8,9,...,2690,2691,2692,2693,2694,2695,2696,2697,2698,2699
rutA,6.323789,6.255952,8.634405,3.410407,8.206440,4.531128,6.282481,4.892818,8.440334,9.081220,...,4.099696,1.682959,6.466848,2.465234,0.780672,5.195210,1.331242,2.123105,0.793510,4.495210
rutR,5.412827,10.803509,7.477550,8.109471,5.802906,9.167125,4.882698,7.191854,5.564458,6.196922,...,2.406741,3.607548,4.669222,2.982365,1.882523,5.564056,4.599585,3.644200,3.930167,4.030351
nemA,6.566084,2.062341,5.699295,6.304175,5.783085,4.620637,4.425961,4.537815,4.728422,5.464784,...,2.347598,1.481209,2.081371,2.394759,1.044559,1.512730,1.009317,1.649925,0.454898,0.334220
fepB,4.315959,3.220587,1.436707,4.618057,1.886708,2.179786,3.993062,5.181278,2.264841,1.996795,...,0.970205,4.468724,0.863026,2.101162,0.070177,1.042788,4.719908,1.652233,1.166358,1.584056
rutG,4.808177,4.654472,2.742947,7.873641,4.419461,3.770729,6.487468,8.139107,3.852718,4.285857,...,5.105571,1.567304,1.704570,3.781724,1.035974,1.233056,1.823780,5.906062,0.602321,0.512264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
artJ,2.964827,1.816167,0.449186,2.123357,0.677656,2.280476,2.225141,1.558472,0.726116,1.997332,...,1.596514,2.342552,3.428169,3.898952,2.731249,5.046719,2.596797,1.934258,3.393242,4.389833
rplD,3.159589,3.441790,5.414819,4.475098,3.932685,5.873825,1.689953,2.873772,3.405268,4.375065,...,2.144228,3.221491,6.676570,3.395882,7.937324,1.961311,3.230171,2.073238,3.727930,1.332269
lldR,6.658100,3.123296,4.047340,4.236098,3.666389,6.281462,6.056645,4.392042,2.814379,5.051746,...,7.835859,0.818901,0.989346,5.880452,2.869600,5.172103,2.266236,6.714973,4.614030,3.603412
potH,11.234700,16.312021,11.429543,8.585354,10.745720,14.108986,13.763955,6.708334,8.874323,11.360719,...,12.536671,2.381887,5.788811,10.701077,10.950673,14.082977,3.286100,12.023819,13.277557,4.154403


In [9]:
def read_dataset(dataset_name,idx2gene):
    ds = pd.read_csv(dataset_name,index_col=0)
    return ds.rename(index=idx2gene)

    
    
    
    
ds = read_dataset(os.path.join(ecoli100, 'simulated_noNoise_0.csv'), idx2gene)
ds.to_csv(os.path.join(ecoli100, 'simulated_noNoise_0.genes.csv'))

In [14]:
dsn = ds.divide(ds.max())
dsn

,0,1,2,3,4,5,6,7,8,9,...,2690,2691,2692,2693,2694,2695,2696,2697,2698,2699
rutA,0.393296,0.272449,0.462904,0.193306,0.475479,0.228981,0.412399,0.274334,0.508305,0.471645,...,0.269959,0.089786,0.426911,0.151509,0.055443,0.368900,0.071672,0.131799,0.052238,0.311843
rutR,0.336641,0.470497,0.400883,0.459654,0.336219,0.463262,0.320513,0.403238,0.335110,0.321845,...,0.158480,0.192463,0.308240,0.183291,0.133696,0.395091,0.247635,0.226227,0.258727,0.279594
nemA,0.408365,0.089816,0.305548,0.357328,0.335071,0.233505,0.290532,0.254429,0.284761,0.283821,...,0.154586,0.079023,0.137402,0.147177,0.074184,0.107416,0.054340,0.102425,0.029946,0.023186
fepB,0.268423,0.140258,0.077024,0.261757,0.109315,0.110156,0.262115,0.290507,0.136396,0.103706,...,0.063886,0.238407,0.056973,0.129133,0.004984,0.074046,0.254113,0.102568,0.076783,0.109889
rutG,0.299036,0.202704,0.147054,0.446287,0.256063,0.190555,0.425855,0.456349,0.232023,0.222591,...,0.336194,0.083616,0.112528,0.232418,0.073574,0.087556,0.098190,0.366640,0.039651,0.035537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
artJ,0.184392,0.079095,0.024082,0.120354,0.039263,0.115244,0.146064,0.087381,0.043729,0.103734,...,0.105128,0.124975,0.226311,0.239622,0.193972,0.358356,0.139808,0.120076,0.223381,0.304532
rplD,0.196505,0.149891,0.290297,0.253654,0.227859,0.296835,0.110933,0.161129,0.205076,0.227225,...,0.141194,0.171867,0.440756,0.208705,0.563704,0.139268,0.173908,0.128704,0.245414,0.092422
lldR,0.414088,0.136021,0.216984,0.240107,0.212430,0.317435,0.397574,0.246256,0.169491,0.262369,...,0.515979,0.043688,0.065312,0.361402,0.203797,0.367259,0.122011,0.416856,0.303747,0.249977
potH,0.698721,0.710395,0.612756,0.486628,0.622605,0.713000,0.903503,0.376127,0.534441,0.590034,...,0.825520,0.127074,0.382150,0.657668,0.777711,1.000000,0.176919,0.746422,0.874077,0.288200


In [18]:
from scipy.special import logit
ldsn = logit(dsn)
ldsn.to_csv(os.path.join(ecoli100, 'simulated_noNoise_0.genes.logit.csv'))
ldsn

,0,1,2,3,4,5,6,7,8,9,...,2690,2691,2692,2693,2694,2695,2696,2697,2698,2699
rutA,-0.433477,-0.982231,-0.148658,-1.428670,-0.098161,-1.214072,-0.354058,-0.972744,0.033221,-0.113543,...,-0.994832,-2.316252,-0.294466,-1.722817,-2.835363,-0.536939,-2.561283,-1.885141,-2.898300,-0.791519
rutR,-0.678300,-0.118147,-0.401787,-0.161734,-0.680189,-0.147216,-0.751414,-0.391992,-0.685163,-0.745305,...,-1.669580,-1.434086,-0.808361,-1.494211,-1.868670,-0.425962,-1.111265,-1.229740,-1.052594,-0.946475
nemA,-0.370727,-2.315886,-0.821016,-0.586980,-0.685339,-1.188626,-0.892802,-1.075127,-0.920966,-0.925589,...,-1.699078,-2.455702,-1.837036,-1.756913,-2.524127,-2.117416,-2.856619,-2.170565,-3.477939,-3.740768
fepB,-1.002638,-1.813149,-2.483485,-1.036856,-2.097753,-2.089150,-1.035003,-0.892921,-1.845551,-2.156707,...,-2.684629,-1.161435,-2.806520,-1.908642,-5.296538,-2.526138,-1.076795,-2.169007,-2.486885,-2.091871
rutG,-0.851894,-1.369478,-1.757899,-0.215683,-1.066535,-1.446411,-0.298784,-0.175049,-1.196922,-1.250628,...,-0.680302,-2.394204,-2.065178,-1.194710,-2.533038,-2.343842,-2.217503,-0.546657,-3.187168,-3.301000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
artJ,-1.486871,-2.454709,-3.701933,-1.989078,-3.197412,-2.038257,-1.765810,-2.346035,-3.085027,-2.156407,...,-2.141502,-1.946137,-1.229257,-1.154751,-1.424406,-0.582507,-1.816887,-1.991709,-1.246071,-0.825807
rplD,-1.408284,-1.735453,-0.893943,-1.079219,-1.220441,-0.862416,-2.081246,-1.649855,-1.354864,-1.224049,...,-1.805407,-1.572455,-0.238096,-1.332752,0.256210,-1.821381,-1.558181,-1.912468,-1.123223,-2.284410
lldR,-0.347090,-1.848740,-1.283328,-1.152092,-1.310341,-0.765585,-0.415583,-1.118681,-1.589238,-1.033692,...,0.063939,-3.086002,-2.661038,-0.569286,-1.362728,-0.543993,-1.973524,-0.335692,-0.829519,-1.098737
potH,0.841217,0.897306,0.458910,-0.053500,0.500618,0.910001,2.236764,-0.506018,0.137981,0.364104,...,1.554206,-1.927083,-0.480433,0.652920,1.252375,inf,-1.537363,1.079621,1.937495,-0.904142
